Choose TASK, MODEL and TYPE

In [0]:
TASK = 'tweetsent'
MODEL = 'bert'
TYPE = 'large'

Get mt-dnn repo and requirements needed for colab

In [70]:
!git clone https://github.com/namisan/mt-dnn
%cd mt-dnn 
!git checkout 60aa9dc4ec1
!pip install pytorch_pretrained_bert==0.5.1
!pip install sentencepiece 
!pip install seqeval  
!pip install tensorboardX
!pip install transformers

Cloning into 'mt-dnn'...
remote: Enumerating objects: 551, done.
remote: Counting objects: 100% (551/551), done.
remote: Compressing objects: 100% (323/323), done.
remote: Total 2132 (delta 259), reused 467 (delta 212), pack-reused 1581
Receiving objects: 100% (2132/2132), 1.70 MiB | 5.08 MiB/s, done.
Resolving deltas: 100% (1152/1152), done.
/content/mt-dnn_port/mt-dnn
Note: checking out '60aa9dc4ec1'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 60aa9dc support F1 mac/mic


Apply patches

In [71]:
!patch data_utils/metrics.py < ../utils/patch/metrics.patch
!patch mt_dnn/model.py < ../utils/patch/model.patch
!patch prepro_std.py < ../utils/patch/prepro_std.patch
!patch train.py < ../utils/patch/train.patch

patching file data_utils/metrics.py
patching file mt_dnn/model.py
patching file prepro_std.py
patching file train.py


Get spefics args

In [0]:
from subprocess import run

INPUT_EN = "../data/input/en"
INPUT_PT = "../data/input/pt"

PREPRO_BERT = f"--do_lower_case --root_dir {INPUT_EN} --model bert-{TYPE}-uncased"
PREPRO_MULTILINGUAL = f"--model bert-{TYPE}-multilingual-cased --root_dir {INPUT_PT}"
PREPRO_BERT_PT = f"--model neuralmind/bert-{TYPE}-portuguese-cased --root_dir {INPUT_PT}"

TRAIN_MT_DNN = f"--init_checkpoint mt_dnn_models/mt_dnn_{TYPE}_uncased.pt \
              --data_dir {INPUT_EN}/bert_{TYPE}_uncased_lower/"

TRAIN_BERT = f"--init_checkpoint bert-{TYPE}-uncased \
               --data_dir {INPUT_EN}/bert_{TYPE}_uncased_lower/"

TRAIN_MULTILINGUAL = f"--data_dir  {INPUT_PT}/bert_{TYPE}_cased \
                  --init_checkpoint bert-{TYPE}-multilingual-cased"

TRAIN_BERT_PT = f"--data_dir  {INPUT_PT}/bert_{TYPE}_cased \
                  --init_checkpoint neuralmind/bert-{TYPE}-portuguese-cased"

if  MODEL == "bert":
   PREPRO = PREPRO_BERT
   TRAIN = TRAIN_BERT
elif MODEL ==  "mt-dnn":
   print("running mt-dnn download script ...wait")
   run(["bash", "download.sh"])
   PREPRO = PREPRO_BERT
   TRAIN = TRAIN_MT_DNN
elif MODEL == "bert-pt":
   PREPRO = PREPRO_BERT_PT
   TRAIN = TRAIN_BERT_PT
elif MODEL == "bert-multilingual":
   PREPRO = PREPRO_MULTILINGUAL
   TRAIN = TRAIN_MULTILINGUAL
else:
  raise ValueError('Invalid model option')

Preprocess data

In [83]:
!python prepro_std.py --task_def ../data/task-def/{TASK}.yaml {PREPRO}
!python train.py  --task_def ../data/task-def/{TASK}.yaml --train_datasets {TASK} --test_datasets {TASK} --tensorboard {TRAIN}

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
2020-04-17 19:01:33.835918: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
04/17/2020 07:01:35 Task tweetsent
04/17/2020 07:01:35 ../data/input/en/bert_base_uncased_lower/tweetsent_train.json
04/17/2020 07:01:39 ../data/input/en/bert_base_uncased_lower/tweetsent_test.json
2020-04-17 19:01:42.272396: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
Namespace(adam_eps=1e-06, answer_att_hidden_size=128, answer_att_type='bilinear', answer_dropout_p=0.1, answer_mem_drop_p=0.1, answer_mem_type=1, answer_merge_opt=1, answer_num_turn=5, answer_opt=0, answer_rnn_type='gru', answer_sum_att_type='bilinear', answer_weight_norm_on=False, batch_size=8, batch_size_eval=8, bert_dropout_p

Show test results

In [0]:
import json

filepath = 'checkpoint/{}_test_scores_4.json'.format(TASK) 
with open(filepath) as f:
    scores = json.load(f)

print(scores['metrics'])

{'Pearson': 85.01105592648992, 'MSE': 13.714753768382353}
